In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [3]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [4]:
# 블로그 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [5]:
lis = driver.find_elements_by_css_selector('li.bx._svp_item')
len(lis)

110

In [6]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text().strip()
title

AttributeError: 'WebElement' object has no attribute 'select_one'

In [ ]:
date = li.select_one('.sub_time.sub_txt').get_text().strip()
date

'2021.11.19.'

In [ ]:
lis[1].select_one('.sub_time.sub_txt').get_text().strip()

'6일 전'

In [ ]:
desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
desc

'지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배기부터 고등어조림,쌈밥 등 풍성한 음식에 아름다운 바다풍경까지 감상할 수 있었던 식당으로 만족스러웠던 곳이랍니다. 1....'

In [ ]:
href = li.select_one('.api_txt_lines.total_tit._cross_trigger')['href']
href

'https://blog.naver.com/tok4/222573156818'

- 날짜처리

In [ ]:
from datetime import datetime, timedelta
today = datetime.now()  ##datetime.today() 와 같음
today

datetime.datetime(2021, 12, 8, 10, 44, 55, 922624)

In [ ]:
today.year, today.month, today.day

(2021, 12, 8)

In [ ]:
# 어제
date=today-timedelta(days=1)  ## today-timedata(1)과 같다
f'{date.year}.{date.month}.{date.day}.'

'2021.12.7.'

In [ ]:
# 6일전
date=today-timedelta(days=6)  ## today-timedata(1)과 같다
f'{date.year}.{date.month}.{date.day}.'

'2021.12.2.'

In [ ]:
# 13시간 전
date=today-timedelta(hours=13)
f'{date.year}.{date.month}.{date.day}.'

'2021.12.7.'

In [ ]:
# 45분전
date=today-timedelta(minutes=45)
f'{date.year}.{date.month:02d}.{date.day:02d}.'


'2021.12.08.'

In [ ]:
def convert_date(s):
    today=datetime.now()
    if '어제' in s :
        x = today-timedelta(days=1)
    if '일' in s and len(s) ==4:
        x=today-timedelta(days=int(str(s)[0]))
    elif '일' in s and len(s) ==5:
        x=today-timedelta(days=int(str(s)[0:2]))
    if '시간' in s and len(s) ==5:
        x=today-timedelta(hours=int(str(s)[0]))
    elif '시간' in s and len(s) ==6:
        x=today-timedelta(hours=int(str(s)[0:2]))
    if '분' in s and len(s) ==4:
        x=today-timedelta(minutes=int(str(s)[0]))
    elif '분' in s and len(s) ==5:
        x=today-timedelta(minutes=int(str(s)[0:2]))
    elif '분' in s and len(s) ==6:
        x=today-timedelta(minutes=int(str(s)[0:3]))

    return f'{x.year}.{x.month:02d}.{x.day:02d}.{x.hour:02d}.{x.minute:02d}'


In [ ]:
convert_date('115분 전')

'2021.12.08.09.38'

In [ ]:
def convert_date(s):
    if len(s) == 11:
        return s[:-1]
    today=datetime.now()
    if '어제' in s :
        x = today-timedelta(days=1)
    elif '일' in s :
        x=today-timedelta(days=int(s[:-3]))
    elif '시간' in s:
        x=today-timedelta(hours=int(s[:-4]))
    elif '분' in s :
        x=today-timedelta(minutes=int(s[:-3]))

    return f'{x.year}.{x.month:02d}.{x.day:02d}.{x.hour:02d}.{x.minute:02d}'

- 반복문으로 모든 데이터 가져오기

In [ ]:
lines=[]
for li in lis:
    title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    desc = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    href = li.select_one('.api_txt_lines.total_tit._cross_trigger')['href']
    lines.append([title,date,desc,href])

In [ ]:
df=pd.DataFrame(lines, columns=['글제목','게시날짜','게시내용','블로그주소'])
df

,글제목,게시날짜,게시내용,블로그주소
0,만족뿐인 제주 맛집 공유,2021.11.19.,지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...,https://blog.naver.com/tok4/222573156818
1,또 생각났던 제주 맛집,6일 전,제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...,https://cafe.naver.com/ungsangstory/940466
2,북적였던 제주 맛집 리스트,2021.11.13.,무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...,https://blog.naver.com/caucommando/222564796496
3,감격했던 제주 맛집 소개,2021.11.15.,얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...,https://blog.naver.com/topnotchlady/222568948674
4,제주도 가볼만한곳 제주 맛집,7일 전,제주도 가볼만한곳 제주 맛집은 깔끔한 메뉴가 푸짐하게 나오기로 소문 난 곳이라 부담...,https://cafe.naver.com/gangmok/1543161
...,...,...,...,...
105,제쇼 12월 제주맛집 공유상 진행합니다.^^,2일 전,[제쇼 12월 제주맛집 공유상 ] 회원님들의 제주맛집 또는 제주카페 정보를 공유해주...,https://cafe.naver.com/lovejeju24/72363
106,만족했던 소노벨 제주 맛집,2021.10.19.,맛도 퀄리티도 좋았던 곳이라 평소에 족발이나 보쌈 좋아하셨다면 소노벨 제주 맛집 여...,https://cafe.naver.com/cosmania/33406306
107,[제주맛집] 사장님 귀는 당나귀 귀 정호영쉐프가 운영하는 제주 교래리 우동카덴,2021.11.27.,그렇지만 아주 특별한 맛은 아닌것 같음. 우동 그릇이 엄청 크고 무거움(세수대야 수...,https://cafe.naver.com/lovejeju24/72006
108,풍요로운 제주 맛집 추천,2021.08.15.,이어서 등장만으로도 이목을 집중시킨 제주 맛집 추천의 제주달빛좋은밤이 야심차게 마련...,https://cafe.naver.com/gangmok/1506627


In [ ]:
df.to_csv('네이버_제주맛집.csv', index=False)

In [ ]:
driver.close()